## Assignment 5

*100 points (8% of course grade)*</br>
*Assigned: Tue, July 16th*</br>
**Due: Mon, July 29th, 23:59**

This homework should be done in parts as soon as relevant topics are covered in lectures. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically, including this Notebook (A5.ipynb). Please follow the submission instructions for each problem carefully.

In this assignment, you need to solve three problems. In Problem 1, you will learn to create indexes to speed up query performance. In Problem 2, you will answer questions on some query processing algorithms. In Problem 3, you will answer questions on cardinality estimation.

### Setup your PostgreSQL

You will need this setup to create a database on your machine and to test your queries. Please follow our [setup instructions](https://canvas.sfu.ca/courses/84701/pages/postgresql-setup-instruction-for-assignment-2) on Canvas.

### Problem 1: Indexing (40%)

In this task, you will be asked to i) select suitable indexes to speed up query performance and ii) examine the query plan of an SQL query. 

We are going to use a new database called `flights` attached in A5.zip. In the database, there is a single table, called flights. The following shows its schema:

**flights** (fid, year, month_id, day_of_month, day_of_week_id, carrier_id, flight_num, origin_city, origin_state, dest_city, dest_state, departure_delay, taxi_out, arrival_delay, canceled, actual_time, distance)

Note that this task only needs to use four attributes: `carrier_id`, `origin_city`, `actual_time`, and `dest_city`.

Follow the steps to create and load the flights database: 
1. In your terminal, enter the `A5` folder (where the `flight_pg.sql` locates)
2. Create a database named flight (`createdb -U [your username] flights`)
3. Run command: `psql -U [your username] -d flight -f flight_pg.sql`
4. You may also import it with PgAdmin.
5. It may take a long time to insert all data. You may see errors because some columns have empty values, but it's fine.

Consider the following queries:

```sqlite
(Q1): select distinct carrier_id
     from flights
     where origin_city = 'seattle wa' and actual_time <= 180;
```
```sqlite
(Q2): SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;
```
```sqlite
(Q3): SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Seattle WA' AND actual_time <= 30;
```

##### (a). Choose one single simple index (index on **one attribute**) that is most likely to speed up all three queries. Write down the CREATE INDEX statement. (5 points)

In [ ]:
CREATE INDEX city_index ON Flights (origin_city)

##### (b). Explain why you chose that index in (a). (3 points)
If you look at the time a flight takes as an index, you will have a potentially large number of flights to scroll through.
If you look at the city the flight originates from as an index, you will have fewer cities (compared to the first index)
which you would need to scan through. Ask professor if this reasoning must be refined.

##### (c). Open a command line shell and start the PostgreSQL. Connect to the flights database, and add the index that you indicate above to the flights table. Please check whether each query used the index or not. Hint: you can use `EXPLAIN QUERY...` (refer to the [documentation link](https://www.postgresql.org/docs/14/using-explain.html)) to see the query plan of each query. (6 points)

```Input your answer in this cell```

* **Did Query (Q1) use the index?**
 Unique  (cost=1485.91..1487.54 rows=22 width=3)
   ->  Sort  (cost=1485.91..1486.72 rows=327 width=3)
         Sort Key: carrier_id
         ->  Bitmap Heap Scan on flights  (cost=7.58..1472.25 rows=327 width=3)
               Recheck Cond: ((origin_city)::text = 'seattle wa'::text)
               Filter: (actual_time <= '180'::double precision)
               ->  Bitmap Index Scan on city_index  (cost=0.00..7.50 rows=410 width=0)
                     Index Cond: ((origin_city)::text = 'seattle wa'::text)
                     * Yes
* **Did Query (Q2) use the index?**
Unique  (cost=1485.91..1487.54 rows=22 width=3)
   ->  Sort  (cost=1485.91..1486.72 rows=327 width=3)
         Sort Key: carrier_id
         ->  Bitmap Heap Scan on flights  (cost=7.58..1472.25 rows=327 width=3)
               Recheck Cond: ((origin_city)::text = 'Gunnison CO'::text)
               Filter: (actual_time <= '180'::double precision)
               ->  Bitmap Index Scan on city_index  (cost=0.00..7.50 rows=410 width=0)
                     Index Cond: ((origin_city)::text = 'Gunnison CO'::text)
                     * Yes
* **Did Query (Q3) use the index?**
Unique  (cost=22626.96..22627.30 rows=21 width=3)
   ->  Sort  (cost=22626.96..22627.13 rows=67 width=3)
         Sort Key: carrier_id
         ->  Bitmap Heap Scan on flights  (cost=251.11..22624.93 rows=67 width=3)
               Recheck Cond: ((origin_city)::text = 'Seattle WA'::text)
               Filter: (actual_time <= '30'::double precision)
               ->  Bitmap Index Scan on city_index  (cost=0.00..251.09 rows=22755 width=0)
                     Index Cond: ((origin_city)::text = 'Seattle WA'::text)
                     * Yes


##### Consider this query:
SELECT DISTINCT F2.origin_city
     FROM Flights F1, Flights F2
     WHERE F1.dest_city = F2.dest_city
         AND F1.origin_city='Gunnison CO'
         AND F1.actual_time <= 30;

```sql
(Q4): SELECT DISTINCT F2.origin_city
     FROM Flights F1, Flights F2
     WHERE F1.dest_city = F2.dest_city
         AND F1.origin_city='Gunnison CO'
         AND F1.actual_time <= 30;
```
##### (d). Choose one simple index (index on **one attribute**), different from the index for the question above, that is likely to speed up this query. Write down the CREATE INDEX statement. (5 points)

In [ ]:
CREATE INDEX destination ON Flights(dest_city);

##### (e). Explain why you chose that index in (d).  (3 points)


If we know the starting and ending city of a flight, it seems more probable that we will have less tuples 
to search through than if we had searched using the starting city and flight time as the index. 
Revise with professor.

#### (f). Connect to the database flights, and check whether the flights table has this second index that you indicate above (Use `\d flights;` in your postgres). If not, add this index to the flights table. Then use the `EXPLAIN` command again to see the plan for Q4.

Did Query (Q4) use this second index? (2 points)

 HashAggregate  (cost=11097.46..11100.59 rows=313 width=13)
   Group Key: f2.origin_city
   ->  Nested Loop  (cost=52.27..11032.20 rows=26105 width=13)
         ->  Bitmap Heap Scan on flights f1  (cost=7.50..1472.17 rows=1 width=13)
               Recheck Cond: ((origin_city)::text = 'Gunnison CO'::text)
               Filter: (actual_time <= '30'::double precision)
               ->  Bitmap Index Scan on city_index  (cost=0.00..7.50 rows=410 width=0)
                     Index Cond: ((origin_city)::text = 'Gunnison CO'::text)
         ->  Bitmap Heap Scan on flights f2  (cost=44.77..9523.46 rows=3657 width=26)
               Recheck Cond: ((dest_city)::text = (f1.dest_city)::text)
               ->  Bitmap Index Scan on destination  (cost=0.00..43.85 rows=3657 width=0)
                     Index Cond: ((dest_city)::text = (f1.dest_city)::text)
                     Yes

##### Now we want to know how effective the two indexes are. We compare the runtimes of the queries with and without indexes. Hint: use `\timing` in Postgres to turn SQL timer on ([Documentation](https://www.postgresql.org/docs/14/pgtesttiming.html)).

##### (g). Execute queries (Q1) - (Q4) on the flights table that do not have the two indexes (or drop the indexes). Please create a screenshot for the runtime of each query and put them in the `runtime` folder in the A5.zip, and name these figures as the img names in the cell below.  (8 points)

<table>
<tr>
    <td> <img src="runtimes/no_indexes_q1.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="runtimes/no_indexes_q2.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="runtimes/no_indexes_q3.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="runtimes/no_indexes_q4.png" alt="Drawing" style="width: 250px;"/> </td>
    </tr>
</table>

##### (h). Execute queries (Q1) - (Q4) on the flights table that has the two indexes. Please create a screenshot for the runtime of each query and put them in the `runtime` folder in the A5.zip, and name these figures as the img names in the cell below.  (8 points)

<table>
<tr>
    <td> <img src="runtimes/indexes_q1.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="runtimes/indexes_q2.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="runtimes/indexes_q3.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="runtimes/indexes_q4.png" alt="Drawing" style="width: 250px;"/> </td>
    </tr>
</table>

### Problem 2. Query Processing Basics (36 points)
Recall the SQL/XML/MongoDB database for PalWorld from Exam II with some modifications. Human "trainers" capture Pals and can place them in production sites. Every Pal belongs to exactly one trainer (no "wild" Pals considered). We have the following tables:
* Trainer (tid, nickname): Each trainer has a unique id and a nickname.
* Pal (pid, species, suitability, level, tid): Each Pal has a unique id and belongs to one of the species. A Pal has exactly one work suitability (like Watering and Handiwork) with a level. Level is an integer in {1,2,3,4}. It has a trainer — tid is a foreign key referencing Trainer(tid).
* Site (sid, type, req_suitability): Each site has a unique sid and a type. Each site requires a work suitability for a Pal to work at this site, i.e., the Pal's suitability must be the same as the site’s req_suitability.
* WorkAt(pid, sid, starttime): Here, pid is a foreign key referencing Pal(pid) and sid is a foreign key referencing Site(sid). A record <pid, sid> means the Pal is currently working at Site from "starttime". A Pal cannot work at multiple Sites at the same time.

For this problem, we have following additional information:
* All rows in each table are stored as compactly as possible on disk blocks using the traditional n-ary storage model
* $|Pal| = 10^5$, all Pal rows are sorted by pid, and each block holds $10$ Pal rows.
* $|Trainer| = 5*10^3$, all Trainer rows are sorted by by tid, and each block holds $20$ Trainer rows.


Answer the following questions. Note that one question may build on another; check each step carefully so you don’t get subsequent questions wrong because of a previous mistake.
* (a) How many blocks does Pal take?
* (b) How many blocks does Trainer take?

```input your answer for 2(a) and 2(b) in this cell:```
a) The Pal table takes 10,000 blocks
b) The trainer table takes 250 blocks

Suppose that we have a total of 21 memory blocks available to sort Pal by tid using the external merge sort algorithm described in lecture. (Any block used for buffering output must come from these blocks too.) For the following questions, if the algorithm already finished in Pass $k$, you should answer "N/A" for the number of level-$k'$ runs for any $k' > k$.
* (c) How many level-0 runs does the algorithm produce?
* (d) How many level-1 runs does the algorithm produce?
* (e) How many level-2 runs does the algorithm produce?
* (f) How many level-3 runs does the algorithm produce?
* (g) How many passes does the algorithm take? (Note that Pass  counts as one pass too.)

```input your answer for 2(c) to 2(g) in this cell:```
Remember: for the level 0 run you should be dividing by 21
All subsequent runs divide by 20
c) 477
d) 24
e) 2 
f) 1
g) 4

Suppose instead we have a total of $M=26$ memory blocks available to perform the two-pass hash join algorithm to compute $Pal \bowtie Trainer$. (Any block used for buffering output must come from these blocks too.) For simplicity, assume that all trainers have an equal number of Pal, and we have a perfect hash function.
* (h) What is the number of partitions per table the partitioning phase creates?
* (i) What is the size of a Pal partition (in blocks)?
* (j) What is the size of a Trainer partition (in blocks)?
* (k) In the probing phase, which table’s partitions (Pal or Trainer) must we use for the two-pass hash join to work?
* (l) Continuing with the above, suppose instead you get to pick $M$, the number of memory blocks available. What is the minimum value of $M$  required for the two-pass hash join algorithm to work?

```input your answer for 2(h) to 2(l) in this cell:```
h) 25
i) 400
j) 10
k) Trainer
l) min(B(Pal), B(Trainer)) <= M^2
250 <= M^2
M >= 15.8
The minimum value of M is 16

### Problem 3: Query Optimization Basics (24 points)

Consider again the four tables from the previous problem. Suppose that:
* $|Pal| = 10^5$, $|\pi_{species} Pal| = 10^2$
* $|Site| = 500$ and $|\pi_{type} Site| = 50$.
* $|WorkAt| = 4000$ and $|\pi_{sid} WorkAt| = 500$.

Using the cardinality estimation techniques described in lecture, estimate the number of rows returned by the following queries.

* (a). $\sigma_{species = 'Anubis'} Pal$

* (b). $(\sigma_{type = 'Wheat\ Plantation'} Site) \bowtie WorkAt$

* (c). $Pal \bowtie WorkAt$.

* (d). $\sigma_{species = 'Anubis'} (Pal \bowtie WorkAt)$. Start with your answer for (c) and assume preservation of value set for $Pal.species$.

* (e). $(\sigma_{species = 'Anubis'} Pal) \bowtie WorkAt$. Start with your answer for (a) and assume preservation of value set for $pid$.

* (f). You may notice that you got different answers for (d) and (e) above. But these two queries are equivalent! Discuss which one you feel to be more realistic.

```input your answer for 3(a) to 3(f) in this cell:```
* (a): 100,000 / 100 = 1,000
* (b): (500 / 50) *  4000 / max(500, 50) = 80
* (c): 100,000 * 4000 / max(100,000, 4000) = 4000
* Pal and WorkAt join on the pid. However, I do not know the number of unique pids that WorkAt has.
* Ask TA on how to do this.
* (d): 4000 / 100 = 40
* (e): 1,000 * 4000 / max(100,000, 4000) = 4
* (f): If there are forty pals of the species Anubis, this may suggest an even distribution of all the pals into 
* WorkAt. However, there is a dearth of information that allows me to argue for either answer. I could say that 
* four is plausible if pals of the species Anubis rarely have the requirements asked for at the site. I could 
* also argue for the opposite. 

## Submission instruction

1. For all problem 1,2,3, answer the questions in the corresponding SQL/Markdown cells

2. Please do not add or remove cells.

3. Compress your A5.ipynb (this file) and your screenshots in `runtime` folder into A5.zip and submit on Canvas.